# Homework 3 Code

In [1]:
# Add import statements here
import csv
import numpy as np
from scipy import stats
import time

In [2]:
# To access files in your Google Drive, run this block and follow the instructions
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# To test if the above block worked, run this block
!ls '/content/gdrive/My Drive/'

In [4]:
# open train data

train_data = []
train_fields = []

with open('/content/gdrive/My Drive/washu/cse417t/CSV_Files/cleveland_train.csv', 'r') as csvfile:
  reader = csv.reader(csvfile)
  train_fields = next(reader)

  for row in reader:
    train_data.append(row)

print('All fields:')
print(train_fields)
print(train_data[1])
print('...', len(train_data), 'rows')

All fields:
['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach', 'exang', 'oldpeak', 'slope', 'ca', 'thal', 'heartdisease::category|0|1']
['37.0', '1.0', '3.0', '130.0', '250.0', '0.0', '0.0', '187.0', '0.0', '3.5', '3.0', '0.0', '3.0', '0']
... 152 rows


In [5]:
# open test data

test_data = []
test_fields = []

with open('/content/gdrive/My Drive/washu/cse417t/CSV_Files/cleveland_test.csv', 'r') as csvfile:
  reader = csv.reader(csvfile)
  test_fields = next(reader)

  for row in reader:
    test_data.append(row)

print('All fields:')
print(test_fields)
print(test_data[1])
print('...', len(test_data), 'rows')

All fields:
['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach', 'exang', 'oldpeak', 'slope', 'ca', 'thal', 'heartdisease::category|0|1']
['67.0', '1.0', '4.0', '160.0', '286.0', '0.0', '2.0', '108.0', '1.0', '1.5', '2.0', '3.0', '3.0', '1']
... 145 rows


 ## Find test error

The `find_test_error` function computes the test error of a linear classifier $w$. 

The hypothesis is assumed to be of the form $sign([1, x(N,:)] \cdot w)$.

Inputs:
* `w` is the weight vector
* `X` is the data matrix (without an initial column of 1's)
* `y` are the data labels (plus or minus 1)

Outputs:
* `test_error` is the binary error of $w$ on the data set $(X, y)$ error; this should be between 0 and 1. 

In [6]:
def find_test_error(w, X, y):
  # Your code here, assign the proper value to test_error:

  error_counter = 0

  for i in range(X.shape[0]):
    # determine sign and check if it is correct
    sign = np.sign(np.dot(w, X[i]))
    if sign != y[i]:
      error_counter = error_counter + 1

  # find average binary error
  test_error = error_counter / X.shape[0]

  print(test_error)

  return test_error

 ## Logistic Regression

The `logistic_reg`  learn a logistic regression model using gradient descent.

Inputs:
* `X` is the data matrix (without an initial column of 1's)
* `y` are the data labels (plus or minus 1)
* `w_init` is the initial value of the w vector ($d+1$ dimensional)
* `max_its` is the maximum number of iterations to run for
* `eta` is the learning rate

Outputs:
* t is the number of iterations gradient descent ran for
* w is the learned weight vector
* e_in is the in-sample (cross-entropy) error 

In [7]:
def logistic_reg(X, y, w_init, max_its, eta):

  # Your code here, assign the proper values to t, w, and e_in:

  # start clocking time
  t0 = time.time()

  #init iterations
  t = 0

  # transpose X to use for calculations
  X_t = X.transpose()

  # stop condition
  small_enough = False

  for i in range(max_its):

    if small_enough == True:
      break

    # gradient calculation: lecture 10 slide 10
    #152, 14 (numerator is point wise, use *)
    numer = np.multiply(X_t, y)
    term1 = np.dot(w_init, X_t)
    term2 = np.multiply(term1, y)
    denom = np.add(1, np.exp(term2))

    avg_gradient = np.multiply(-1, np.divide(np.sum((numer/denom), axis=1), len(X)))

    # update weight
    update = np.multiply(eta, avg_gradient)
    w_init = np.subtract(w_init, update)

    # check magnitude of avg gradient elements
    bool_array = np.abs(avg_gradient) > 10**-3

    if any(bool_array) == False:
      small_enough = True

    # keep track of iterations. for loop breaks once max iters is reached
    t = t + 1

  # finding cross entropy error with learned weight (referred to slide 4 of lecture 10)
  a = np.dot(w_init, X.transpose())
  b = np.multiply(-1, np.multiply(a, y))
  ln_term = np.add(1, np.exp(b))
  sum_error = np.sum(np.log(ln_term))
  e_in = np.divide(sum_error, len(X))

  # stop clock time
  t1 = time.time() - t0

  return t, w_init, e_in, t1

## Run and Plot

Run your code and plot figures below

In [8]:
# Other code here:

In [9]:
# setting up training data and init w of all zeros

X_train = []
Y_train = []

row1 = train_data[1]
x_1 = row1[:-1]
x_1.insert(0,1)
X_train = np.append(X_train, x_1)

for n in range(len(train_data)):
  # 14 dimensional: insert leading 1
  row = train_data[n]
  x_n = row[:-1]
  x_n.insert(0,1)

  x_n = np.array(x_n, dtype=np.float64)

  X_train = np.vstack((X_train, x_n))
  X_train = np.array(X_train, dtype=float)

  y_n = np.float(train_data[n][-1])
  if y_n == 0:
    Y_train = np.append(Y_train, -1)
  else:
    Y_train = np.append(Y_train, 1)

w = np.zeros(14)
X_train = X_train[1:]

print('X: ', X_train)
print(len(X_train), ',', len(X_train[0]))
print('Y: ', Y_train)
print('w: ', w)


X:  [[ 1. 67.  1. ...  2.  2.  7.]
 [ 1. 37.  1. ...  3.  0.  3.]
 [ 1. 56.  1. ...  1.  0.  3.]
 ...
 [ 1. 68.  1. ...  2.  2.  7.]
 [ 1. 57.  1. ...  2.  1.  7.]
 [ 1. 57.  0. ...  2.  1.  3.]]
152 , 14
Y:  [ 1. -1. -1. -1.  1. -1. -1. -1. -1. -1.  1. -1. -1. -1. -1.  1.  1.  1.
 -1. -1.  1.  1. -1.  1. -1.  1.  1. -1. -1. -1. -1. -1.  1.  1. -1.  1.
  1.  1.  1.  1. -1.  1. -1. -1.  1.  1. -1. -1. -1.  1. -1.  1.  1.  1.
  1.  1. -1. -1. -1.  1. -1.  1.  1. -1. -1. -1.  1. -1.  1. -1.  1. -1.
 -1. -1. -1.  1.  1. -1. -1. -1.  1. -1. -1.  1.  1. -1.  1. -1.  1.  1.
  1.  1. -1. -1. -1. -1.  1. -1. -1.  1. -1.  1. -1. -1. -1. -1. -1. -1.
 -1.  1. -1.  1. -1.  1. -1. -1. -1. -1. -1.  1.  1.  1. -1.  1. -1. -1.
 -1. -1. -1. -1. -1. -1.  1.  1.  1.  1. -1.  1. -1. -1. -1.  1.  1. -1.
 -1.  1.  1. -1.  1.  1.  1.  1.]
w:  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [10]:
eta = 10**-5
max_iters_0 = 10**4
max_iters_1 = 10**5
max_iters_2 = 10**6 

t_0, w_0, e_in_0, time_0 = logistic_reg(X_train, Y_train, w, max_iters_0, eta)

print('t_0: ', t_0)
print('w_0: ', w_0)
print('e_in_0: ', e_in_0)
print('time_0: ', time_0)

t_0:  10000
w_0:  [ 0.00032553  0.02316466  0.00587086  0.01547472  0.01904406 -0.00115137
 -0.00093957  0.0045843  -0.02706045  0.00520414  0.01507737  0.00581902
  0.01519724  0.03955954]
e_in_0:  0.5847145522443018
time_0:  0.4607808589935303


In [11]:
t_1, w_1, e_in_1, time_1 = logistic_reg(X_train, Y_train, w, max_iters_1, eta)

print('t_1: ', t_1)
print('w_1: ', w_1)
print('e_in_1: ', e_in_1)
print('time_1: ', time_1)

t_1:  100000
w_1:  [ 0.00052699  0.00682633  0.04066563  0.10819079  0.01244496 -0.00082465
 -0.01090669  0.0394019  -0.02658727  0.0377911   0.10391188  0.03779678
  0.11746569  0.25456501]
e_in_1:  0.49370175927710563
time_1:  3.6568708419799805


In [12]:
t_2, w_2, e_in_2, time_2 = logistic_reg(X_train, Y_train, w, max_iters_2, eta)

print('t_2: ', t_2)
print('w_2: ', w_2)
print('e_in_2: ', e_in_2)
print('time_2: ', time_2)

t_2:  1000000
w_2:  [-0.02465147 -0.0280051   0.13641902  0.32830321  0.01111351 -0.00066102
 -0.11036056  0.22723006 -0.02662668  0.16431093  0.25712227  0.08583061
  0.56050923  0.38035936]
e_in_2:  0.43535260915252544
time_2:  36.89667844772339


In [42]:
# finding test error (and setting up test data to use)

X_test = []
Y_test = []

r1 = test_data[1]
x1 = r1[:-1]
x1.insert(0,1)
X_test = np.append(X_test, x1)

for n in range(len(test_data)):
  # 14 dimensional: insert leading 1
  row = test_data[n]
  x_n = row[:-1]
  x_n.insert(0,1)

  x_n = np.array(x_n, dtype=np.float64)

  X_test = np.vstack((X_test, x_n))
  X_test = np.array(X_test, dtype=float)

  y_n = np.float(test_data[n][-1])
  if y_n == 0:
    Y_test = np.append(Y_test, -1)
  else:
    Y_test = np.append(Y_test, 1)

X_test = X_test[1:]

test_error_0 = find_test_error(w_0, X_test, Y_test)
print('test_error_0: ', test_error_0)

test_error_1 = find_test_error(w_1, X_test, Y_test)
print('test_error_1: ', test_error_1)

test_error_2 = find_test_error(w_2, X_test, Y_test)
print('test_error_2: ', test_error_2)

0.31724137931034485
test_error_0:  0.31724137931034485
0.20689655172413793
test_error_1:  0.20689655172413793
0.1310344827586207
test_error_2:  0.1310344827586207


In [14]:
# scaling each feature in TRAINING DATA:
# (x_i - X_bar) / standard_deviation_X
# zscore of entire column

c1 = []
c2 = []
c3 = []
c4 = []
c5 = []
c6 = []
c7 = []
c8 = []
c9 = []
c10 = []
c11 = []
c12 = []
c13 = []

for i in range(X_train.shape[0]):
  row = X_train[i]

  c1 = np.append(c1, float(row[1]))
  c2 = np.append(c2, float(row[2]))
  c3 = np.append(c3, float(row[3]))
  c4 = np.append(c4, float(row[4]))
  c5 = np.append(c5, float(row[5]))
  c6 = np.append(c6, float(row[6]))
  c7 = np.append(c7, float(row[7]))
  c8 = np.append(c8, float(row[8]))
  c9 = np.append(c9, float(row[9]))
  c10 = np.append(c10, float(row[10]))
  c11 = np.append(c11, float(row[11]))
  c12 = np.append(c12, float(row[12]))
  c13 = np.append(c13, float(row[13]))

c1_zscore = stats.zscore(c1)
c2_zscore = stats.zscore(c2)
c3_zscore = stats.zscore(c3)
c4_zscore = stats.zscore(c4)
c5_zscore = stats.zscore(c5)
c6_zscore = stats.zscore(c6)
c7_zscore = stats.zscore(c7)
c8_zscore = stats.zscore(c8)
c9_zscore = stats.zscore(c9)
c10_zscore = stats.zscore(c10)
c11_zscore = stats.zscore(c11)
c12_zscore = stats.zscore(c12)
c13_zscore = stats.zscore(c13)

X_train_normalized = []
row1 = [1, c1_zscore[1], c2_zscore[1], c3_zscore[1], c4_zscore[1], c5_zscore[1], c6_zscore[1], c7_zscore[1], c8_zscore[1], c9_zscore[1], c10_zscore[1], c11_zscore[1], c12_zscore[1], c13_zscore[1]]
X_train_normalized = np.append(X_train_normalized, row1)

for i in range(len(c1_zscore)):
  row = [1, c1_zscore[i], c2_zscore[i], c3_zscore[i], c4_zscore[i], c5_zscore[i], c6_zscore[i], c7_zscore[i], c8_zscore[i], c9_zscore[i], c10_zscore[i], c11_zscore[i], c12_zscore[i], c13_zscore[i]]
  X_train_normalized = np.vstack((X_train_normalized, row))

X_train_normalized = X_train_normalized[1:]
print(X_train_normalized)


[[ 1.          1.3984336   0.72111026 ...  0.69963115  1.49009161
   1.13265251]
 [ 1.         -1.980929    0.72111026 ...  2.36125514 -0.71960522
  -0.929182  ]
 [ 1.          0.15933398  0.72111026 ... -0.96199283 -0.71960522
  -0.929182  ]
 ...
 [ 1.          1.51107902  0.72111026 ...  0.69963115  1.49009161
   1.13265251]
 [ 1.          0.2719794   0.72111026 ...  0.69963115  0.3852432
   1.13265251]
 [ 1.          0.2719794  -1.38675049 ...  0.69963115  0.3852432
  -0.929182  ]]


In [33]:
# normalizing each feature in TEST DATA using training data mean and standard deviation (since we don't know test data statistics in real life)

col1 = []
col2 = []
col3 = []
col4 = []
col5 = []
col6 = []
col7 = []
col8 = []
col9 = []
col10 = []
col11 = []
col12 = []
col13 = []

train_means = [1]
train_stds = [1]
for i in range(len(X_train.transpose())):
  if i != 0:
    column = X_train.transpose()[i]
    train_means = np.append(train_means, np.mean(column))
    train_stds = np.append(train_stds, np.std(column))

# print(train_means.shape[0])
# print(train_stds)

for i in range(X_test.shape[0]):
  row = X_test[i]

  col1 = np.append(col1, ((float(row[1]) - train_means[1]) / train_stds[1]))
  col2 = np.append(col2, ((float(row[2]) - train_means[2]) / train_stds[2]))
  col3 = np.append(col3, ((float(row[3]) - train_means[3]) / train_stds[3]))
  col4 = np.append(col4, ((float(row[4]) - train_means[4]) / train_stds[4]))
  col5 = np.append(col5, ((float(row[5]) - train_means[5]) / train_stds[5]))
  col6 = np.append(col6, ((float(row[6]) - train_means[6]) / train_stds[6]))
  col7 = np.append(col7, ((float(row[7]) - train_means[7]) / train_stds[7]))
  col8 = np.append(col8, ((float(row[8]) - train_means[8]) / train_stds[8]))
  col9 = np.append(col9, ((float(row[9]) - train_means[9]) / train_stds[9]))
  col10 = np.append(col10, ((float(row[10]) - train_means[10]) / train_stds[10]))
  col11 = np.append(col11, ((float(row[11]) - train_means[11]) / train_stds[11]))
  col12 = np.append(col12, ((float(row[12]) - train_means[12]) / train_stds[12]))
  col13 = np.append(col13, ((float(row[13]) - train_means[13]) / train_stds[13]))

X_test_normalized = []
row_1 = [1, col1[1], col2[1], col3[1], col4[1], col5[1], col6[1], col7[1], col8[1], col9[1], col10[1], col11[1], col12[1], col13[1]]
X_test_normalized = np.append(X_test_normalized, row_1)

for i in range(len(col1)):
  row = [1, col1[i], col2[i], col3[i], col4[i], col5[i], col6[i], col7[i], col8[i], col9[i], col10[i], col11[i], col12[i], col13[i]]
  X_test_normalized = np.vstack((X_test_normalized, row))

X_test_normalized = X_test_normalized[1:]
print(X_test_normalized)
print('-----')
print(X_test)

[[ 1.          0.94785192  0.72111026 ...  2.36125514 -0.71960522
   0.61719388]
 [ 1.          1.3984336   0.72111026 ...  0.69963115  2.59494003
  -0.929182  ]
 [ 1.         -1.53034732 -1.38675049 ... -0.96199283 -0.71960522
  -0.929182  ]
 ...
 [ 1.         -1.19241106  0.72111026 ...  2.36125514 -0.71960522
   0.61719388]
 [ 1.          0.2719794  -1.38675049 ...  0.69963115 -0.71960522
   1.13265251]
 [ 1.         -1.07976564  0.72111026 ...  0.69963115 -0.71960522
   1.13265251]]
-----
[[ 1. 63.  1. ...  3.  0.  6.]
 [ 1. 67.  1. ...  2.  3.  3.]
 [ 1. 41.  0. ...  1.  0.  3.]
 ...
 [ 1. 44.  1. ...  3.  0.  6.]
 [ 1. 57.  0. ...  2.  0.  7.]
 [ 1. 45.  1. ...  2.  0.  7.]]


In [34]:
def log_reg_no_max_its(X, y, w_init, eta):

  # Your code here, assign the proper values to t, w, and e_in:

  # start clocking time
  t0 = time.time()

  #init iterations
  t = 0

  # transpose X to use for calculations
  X_t = X.transpose()

  # stop condition
  small_enough = False

  while small_enough == False:

    if small_enough == True:
      break

    # gradient calculation: lecture 10 slide 10
    #152, 14 (numerator is point wise, use *)
    numer = np.multiply(X_t, y)
    term1 = np.dot(w_init, X_t)
    term2 = np.multiply(term1, y)
    denom = np.add(1, np.exp(term2))

    avg_gradient = np.multiply(-1, np.divide(np.sum((numer/denom), axis=1), len(X)))

    # update weight
    update = np.multiply(eta, avg_gradient)
    w_init = np.subtract(w_init, update)

    # check magnitude of avg gradient elements
    bool_array = np.abs(avg_gradient) >= 10**-6

    if any(bool_array) == False:
      small_enough = True

    # keep track of iterations. for loop breaks once max iters is reached
    t = t + 1

  # finding cross entropy error with learned weight (referred to slide 4 of lecture 10)
  a = np.dot(w_init, X.transpose())
  b = np.multiply(-1, np.multiply(a, y))
  ln_term = np.add(1, np.exp(b))
  sum_error = np.sum(np.log(ln_term))
  e_in = np.divide(sum_error, len(X))

  # stop clock time
  t1 = time.time() - t0

  return t, w_init, e_in, t1

In [35]:
eta_0 = 0.01
eta_1 = 0.1
eta_2 = 1
eta_3 = 4
eta_4 = 5
eta_5 = 6
eta_6 = 7
eta_7 = 7.5
eta_8 = 7.6
eta_9 = 7.65

In [36]:
print('Testing different eta values with changed stop conditions:')
print()
print('eta: 0.01')
t0, w0, e_in0, time0 = log_reg_no_max_its(X_train_normalized, Y_train, w, eta_0)
print('t: ', t0)
print('w: ', w0)
print('e_in: ', e_in0)
print('time0: ', time0)
print()
print('eta: 0.1')
t1, w1, e_in1, time1 = log_reg_no_max_its(X_train_normalized, Y_train, w, eta_1)
print('t: ', t1)
print('w: ', w1)
print('e_in: ', e_in1)
print('time1: ', time1)
print()
print('eta: 1')
t2, w2, e_in2, time2 = log_reg_no_max_its(X_train_normalized, Y_train, w, eta_2)
print('t: ', t2)
print('w: ', w2)
print('e_in: ', e_in2)
print('time2: ', time2)
print()
print('eta: 4')
t3, w3, e_in3, time3 = log_reg_no_max_its(X_train_normalized, Y_train, w, eta_3)
print('t: ', t3)
print('w: ', w3)
print('e_in: ', e_in3)
print('time3: ', time3)
print()
print('eta: 5')
t4, w4, e_in4, time4 = log_reg_no_max_its(X_train_normalized, Y_train, w, eta_4)
print('t: ', t4)
print('w: ', w4)
print('e_in: ', e_in4)
print('time4: ', time4)
print()
print('eta: 6')
t5, w5, e_in5, time5 = log_reg_no_max_its(X_train_normalized, Y_train, w, eta_5)
print('t: ', t5)
print('w: ', w5)
print('e_in: ', e_in5)
print('time5: ', time5)
print()
print('eta: 7')
t6, w6, e_in6, time6 = log_reg_no_max_its(X_train_normalized, Y_train, w, eta_6)
print('t: ', t6)
print('w: ', w6)
print('e_in: ', e_in6)
print('time6: ', time6)
print()
print('eta: 7.5')
t7, w7, e_in7, time7 = log_reg_no_max_its(X_train_normalized, Y_train, w, eta_7)
print('t: ', t7)
print('w: ', w7)
print('e_in: ', e_in7)
print('time7: ', time7)
print()
print('eta: 7.6')
t8, w8, e_in8, time8 = log_reg_no_max_its(X_train_normalized, Y_train, w, eta_8)
print('t: ', t8)
print('w: ', w8)
print('e_in: ', e_in8)
print('time8: ', time8)
print()
print('eta: 7.65')
t9, w9, e_in9, time9 = log_reg_no_max_its(X_train_normalized, Y_train, w, eta_9)
print('t: ', t9)
print('w: ', w9)
print('e_in: ', e_in9)
print('time9: ', time9)
print()

Testing different eta values with changed stop conditions:

eta: 0.01
t:  23222
w:  [-0.35027354 -0.16895917  0.43466056  0.55150731  0.45320422  0.11750064
 -0.32456249  0.32087401 -0.30951214  0.30614969  0.13299168  0.2729831
  0.94903171  0.51505414]
e_in:  0.40738144126659726
time0:  0.8545124530792236

eta: 0.1
t:  2319
w:  [-0.35027354 -0.16895927  0.43466068  0.55150735  0.4532043   0.11750073
 -0.32456251  0.320874   -0.30951218  0.30614972  0.13299158  0.27298317
  0.94903185  0.51505406]
e_in:  0.4073814412659671
time1:  0.0926053524017334

eta: 1
t:  229
w:  [-0.35027359 -0.16896042  0.43466226  0.55150787  0.45320528  0.11750191
 -0.32456283  0.32087391 -0.30951256  0.30615015  0.13299032  0.27298404
  0.94903358  0.515053  ]
e_in:  0.40738144125838255
time2:  0.010398387908935547

eta: 4
t:  53
w:  [-0.35027366 -0.16896184  0.43466479  0.55150865  0.45320651  0.11750364
 -0.32456318  0.32087374 -0.30951299  0.30615078  0.13298888  0.27298503
  0.94903573  0.51505123]
e_in

In [41]:
print('Test errors with changing etas:')
test_error0 = find_test_error(w0, X_test_normalized, Y_test)
print('test_error0: ', test_error0)
print()
test_error1 = find_test_error(w1, X_test_normalized, Y_test)
print('test_error1: ', test_error1)
print()
test_error2 = find_test_error(w2, X_test_normalized, Y_test)
print('test_error2: ', test_error2)
print()
test_error3 = find_test_error(w3, X_test_normalized, Y_test)
print('test_error3: ', test_error3)
print()
test_error4 = find_test_error(w4, X_test_normalized, Y_test)
print('test_error4: ', test_error4)
print()
test_error5 = find_test_error(w5, X_test_normalized, Y_test)
print('test_error5: ', test_error5)
print()
test_error6 = find_test_error(w6, X_test_normalized, Y_test)
print('test_error6: ', test_error6)
print()
test_error7 = find_test_error(w7, X_test_normalized, Y_test)
print('test_error7: ', test_error7)
print()
test_error8 = find_test_error(w8, X_test_normalized, Y_test)
print('test_error8: ', test_error8)
print()
test_error9 = find_test_error(w9, X_test_normalized, Y_test)
print('test_error9: ', test_error9)

Test errors with changing etas:
0.1103448275862069
test_error0:  0.1103448275862069

0.1103448275862069
test_error1:  0.1103448275862069

0.1103448275862069
test_error2:  0.1103448275862069

0.1103448275862069
test_error3:  0.1103448275862069

0.1103448275862069
test_error4:  0.1103448275862069

0.1103448275862069
test_error5:  0.1103448275862069

0.1103448275862069
test_error6:  0.1103448275862069

0.1103448275862069
test_error7:  0.1103448275862069

0.1103448275862069
test_error8:  0.1103448275862069

0.1103448275862069
test_error9:  0.1103448275862069
